# Ce code extrait les festivals les plus consultés depuis le site "infoconcert.com".
##  Il récupère d'abord la page web contenant les informations sur les festivals.
## Ensuite, il extrait le classement, le nom et le lieu de chaque festival à l'aide de BeautifulSoup.
## Puis les données sont nettoyées, doublons éliminés et stockées dans un DataFrame Pandas.
## Ensuite, le lieu de chaque festival est géocodé pour obtenir ses coordonnées géographiques (latitude, longitude).
## Enfin, les festivals avec des coordonnées valides sont conservés et affichés dans le DataFrame final.


In [1]:
!pip install geopy

# Importation des bibliothèques

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import os
from IPython.display import IFrame
import time

# Chargement de la page web contenant les 50 festivals les plus populaires

In [ ]:
# URL de la page
url = "https://infoconcert.com/festival/les-plus-consultes.html"

# Envoyer une requête HTTP
response = requests.get(url)
if response.status_code == 200:
    print("Page chargée avec succès")
else:
    print(f"Erreur lors du chargement de la page : {response.status_code}")
    exit()

Page chargée avec succès


# Extraction des données des festivals à partir de la page

In [ ]:
from bs4 import BeautifulSoup

# Parser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Trouver les sections correspondant aux festivals
festival_rows = soup.find_all("div", class_="row")

# Extraire les données
festivals = []
seen = set()  # Pour éviter les doublons
for row in festival_rows:
    try:
        # Classement
        ranking_tag = row.find("span", class_="top-position-number")
        ranking = ranking_tag.text.strip() if ranking_tag else None

        # Nom du festival
        name_tag = row.find("div", class_="top-line-name").find("a")
        name = name_tag.text.strip() if name_tag else None
        link = f"https://infoconcert.com{name_tag['href']}" if name_tag else None

        # Lieu
        location_tag = row.find("div", class_="top-coming-concerts")
        location = location_tag.text.strip() if location_tag else None

        # Vérifier que les données sont complètes et uniques
        if name and ranking and (ranking, name) not in seen:
            festivals.append({
                "Classement": ranking,
                "Nom": name,
                "Lieu": location,
                "Lien": link
            })
            seen.add((ranking, name))  # Marquer comme vu
    except Exception as e:
        print(f"Erreur lors de l'extraction d'une ligne : {e}")

# Convertir en DataFrame
df = pd.DataFrame(festivals)

# Supprimer les doublons au cas où
df = df.drop_duplicates()

# Afficher un aperçu des données
print("Aperçu des festivals extraits :")
print(df.head())

Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Aperçu des festivals extraits :
  Classement                          Nom                           Lieu  \
0         01            FESTIVAL DE NIMES                    Nimes\n(30)   
1         02       FESTIVAL PAUSE GUITARE                     Albi\n(81)   
2         03  FRANCOFOLIES DE LA ROCHELLE              La Rochelle\n(17)   
3         04        PRINTEMPS DE PEROUGES  Saint Maurice de Remens\n(01)   
4         05        FESTIVAL RETRO C TROP                 Tilloloy\n(80)   

                                                Lien  
0  https://infoconcert.com/festival/

# Nettoyage de la colonne "Lieu"

In [ ]:
# Nettoyer la colonne "Lieu"
df['Lieu'] = df['Lieu'].str.replace(r'\n\(.*?\)', '', regex=True).str.strip()

# Vérifier le résultat
print("Lieux nettoyés :")
print(df[['Classement', 'Nom', 'Lieu']].head())

Lieux nettoyés :
  Classement                          Nom                     Lieu
0         01            FESTIVAL DE NIMES                    Nimes
1         02       FESTIVAL PAUSE GUITARE                     Albi
2         03  FRANCOFOLIES DE LA ROCHELLE              La Rochelle
3         04        PRINTEMPS DE PEROUGES  Saint Maurice de Remens
4         05        FESTIVAL RETRO C TROP                 Tilloloy


# Ajout des coordonnées géographiques des festivals

In [ ]:
# Initialiser le géocodeur avec un délai
geolocator = Nominatim(user_agent="festival_scraper", timeout=10)

def geocode_location(location):
    try:
        print(f"Geocoding : {location}")
        geo = geolocator.geocode(location)
        if geo:
            return geo.latitude, geo.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erreur de géocodage pour {location}: {e}")
        return None, None

# Ajouter un délai entre les requêtes
df[['Latitude', 'Longitude']] = df['Lieu'].apply(
    lambda x: pd.Series(geocode_location(x) if pd.notna(x) else (None, None))
)
time.sleep(1)  # Délai pour éviter d'être bloqué par l'API

# Filtrer les résultats valides
df = df.dropna(subset=['Latitude', 'Longitude'])
print("Festivals avec coordonnées géographiques :")
print(df[['Nom', 'Lieu', 'Latitude', 'Longitude']].head())

Geocoding : Nimes
Geocoding : Albi
Geocoding : La Rochelle
Geocoding : Saint Maurice de Remens
Geocoding : Tilloloy
Geocoding : Luxey
Geocoding : Saint Etienne
Geocoding : Le Barcares
Geocoding : Nancy
Geocoding : Vienne
Geocoding : Gignac
Geocoding : Le Poire Sur Vie
Geocoding : Carcassonne
Geocoding : Carhaix Plouguer
Geocoding : Lyon
Geocoding : Thonon Les Bains
Geocoding : Aix Les Bains
Geocoding : Luxembourg
Geocoding : Enghien
Geocoding : Marciac
Geocoding : Ruoms
Geocoding : Yzeures sur Creuse
Geocoding : Manosque
Geocoding : Montauban
Geocoding : Carhaix Plouguer
Geocoding : Saint Cloud
Geocoding : Clisson
Geocoding : Lyon
Geocoding : Aulnoye Aymeries
Geocoding : Paris
Geocoding : Saint Julien En Genevois
Geocoding : Dijon
Geocoding : Bourges
Geocoding : Marmande
Geocoding : Nort sur Erdre
Geocoding : Arras
Geocoding : Nyon
Geocoding : Camaret Sur Mer
Geocoding : Marseille
Geocoding : Clermont Ferrand
Geocoding : Argeles Sur Mer
Geocoding : Aussonne
Geocoding : Bobital
Geocodin

# Sauvegarde des données dans un fichier CSV

In [6]:
df.to_csv("50_festivals_geolocalises.csv", index=False, encoding="utf-8")
print("Les données ont été sauvegardées dans '50_festivals_geolocalises.csv'.")

Les données ont été sauvegardées dans '50_festivals_geolocalises.csv'.
